In [1]:
import networkx as nx
import copy
import gurobipy as gp
from gurobipy import GRB
from helpers import *

# Instantiate processor

proc = Processor(4, 2, "HotspotConfiguration", 318.15)

# proc_cores is the number of processors per layer.
proc_layers = proc.nr_layers
proc_cores = proc.proc_per_layer

proc_total = proc.nr_pes
proc_config_path = proc.path_to_config


timestep = 1e-1

T_amb = proc.T_amb

coefficients = proc.coefficients


In [2]:
def readTempsFromInit(path):
    """
    Read temperatures from the initial temperature file and return them as a list.

    Parameters:
    -----------
    path : str
        The file path to the initial temperature file. The path should be relative to `proc_config_path`.

    Returns:
    --------
    list[float]
        A list of temperature values for each PE.
    """
    f = open(f'{proc_config_path}/{path}', 'r')
    temps = []
    lines = list(map(lambda x: float(x.split()[-1]), f.readlines()))
    f.close()
    index = 0
    for _ in range(proc_layers):
        for _ in range(proc_cores):
            temps.append(lines[index])
            index += 1
        index += 1
    return temps

def generate_thermal_impact_coefficients(alpha, nr_trials):
    """
    Generate thermal coefficient matrix for processing elements.
    See Thesis Chapter 3.1 for details

    Parameters:
    -----------
    alpha : float
        The power increment used to perturb the power trace of each processing element.
    nr_trials : int
        The number of trials to run for averaging the thermal impact coefficients.

    Returns:
    --------
    list[list[float]]
        The thermal coefficient matrix.
    """
    coefficient_matrices = []

    for i in range(nr_trials):
        default_trace = [[]]
        for _ in range(proc_total):
            default_trace[0].append(random.randint(10, 90))

        coefficients = [ [0] * proc_total for _ in range(proc_total)]
        
        proc.write_power_trace(default_trace, f'default_thermal.ptrace')

        subprocess.run(construct_hotspot_command('default_thermal.ptrace', steady_file='default_steady.steady', grid_steady_file='default_steady.grid.steady'), cwd=proc_config_path)
        default_temperatures = readTempsFromInit(f'default_steady.steady')

        for j in range(proc_total):
            trace = copy.deepcopy(default_trace)
            trace[0][j] += alpha

            proc.write_power_trace(trace, f'updated_thermal.ptrace')
            subprocess.run(construct_hotspot_command('updated_thermal.ptrace', steady_file='updated_steady.steady', grid_steady_file='updated_steady.grid.steady'), cwd=proc_config_path)
            updated_temps = readTempsFromInit(f'updated_steady.steady')
            for i in range(proc_total):
                coefficients[i][j] = (updated_temps[i] - default_temperatures[i]) / alpha
        coefficient_matrices.append(coefficients)
    
    coefficients = [ [0] * proc_total for _ in range(proc_total)]
    for i in range(proc_total):
        for j in range(proc_total):
            coefficients[i][j] = sum(coefficient_matrices[t][i][j] for t in range(nr_trials)) / nr_trials
    return coefficients

def dump_coefficients(coefficients, output):
    """
    Write thermal coefficient matrix to a specified output file.

    Parameters:
    -----------
    coefficients : list[list[float]]
        The thermal coefficient matrix
    output : str
        The file name to write the coefficients to relative to proc_config_path. 

    """
    f = open(f'{proc_config_path}/{output}', 'w')
    for layer in coefficients:
        for coeff in layer:
            f.write(str(coeff))
            f.write(' ')
        f.write('\n')
    f.close()

coefficients = generate_thermal_impact_coefficients(10, 20)
dump_coefficients(coefficients, 'thermal_coefficients.txt')


Parsing input files...


Simulation Summary:
-------------------------
Ambient at 318.15 K
-------------------------
Heat Sink : 6.90 mm
  conductivity = 400.000000 W/(m-K)
  vol. heat capacity = 3550000.000000 J/(m^3-K)
-------------------------
Heat Spreader : 1.00 mm
  conductivity = 400.000000 W/(m-K)
  vol. heat capacity = 3550000.000000 J/(m^3-K)
-------------------------
Layer 3 : 20.00 um
  conductivity = 4.000000 W/(m-K)
  vol. heat capacity = 4000000.000000 J/(m^3-K)
-------------------------
Layer 2 : 50.00 um
  conductivity = 142.857143 W/(m-K)
  vol. heat capacity = 1750000.000000 J/(m^3-K)
  dissipates power
-------------------------
Layer 1 : 20.00 um
  conductivity = 4.000000 W/(m-K)
  vol. heat capacity = 4000000.000000 J/(m^3-K)
-------------------------
Layer 0 : 50.00 um
  conductivity = 142.857143 W/(m-K)
  vol. heat capacity = 1750000.000000 J/(m^3-K)
  dissipates power
-------------------------


Creating thermal circuit...
Resistances: 
Layer 0: 
Resistance [0][

Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state te

Parsing input files...


Simulation Summary:
-------------------------
Ambient at 318.15 K
-------------------------
Heat Sink : 6.90 mm
  conductivity = 400.000000 W/(m-K)
  vol. heat capacity = 3550000.000000 J/(m^3-K)
-------------------------
Heat Spreader : 1.00 mm
  conductivity = 400.000000 W/(m-K)
  vol. heat capacity = 3550000.000000 J/(m^3-K)
-------------------------
Layer 3 : 20.00 um
  conductivity = 4.000000 W/(m-K)
  vol. heat capacity = 4000000.000000 J/(m^3-K)
-------------------------
Layer 2 : 50.00 um
  conductivity = 142.857143 W/(m-K)
  vol. heat capacity = 1750000.000000 J/(m^3-K)
  dissipates power
-------------------------
Layer 1 : 20.00 um
  conductivity = 4.000000 W/(m-K)
  vol. heat capacity = 4000000.000000 J/(m^3-K)
-------------------------
Layer 0 : 50.00 um
  conductivity = 142.857143 W/(m-K)
  vol. heat capacity = 1750000.000000 J/(m^3-K)
  dissipates power
-------------------------


Creating thermal circuit...
Resistances: 
Layer 0: 
Resistance [0][

Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state temperatures...
Computing steady-state te